> Now, given a vector $u^n=[0,1,2,3,4,5]$ we can calculate the values of $u^{n+1}$ by iterating over the values of $u^n$ with a for loop.

现在，给定一个向量$u^n=[0,1,2,3,4,5]$，我们可以通过for循环遍历 $u^n$ 的值来计算 $u^{n+1}$ 的值。

In [1]:
import numpy

In [2]:
u = numpy.array((0,1,2,3,4,5))

for i in range (1,len(u)):
    print(u[i]-u[i-1])

1
1
1
1
1


> This is the expected result and the execution time was nearly instantaneous. If we perform the same operation as an array operation, then rather than calculate $u^n_i−u^n_{i−1}$ 5 separate times, we can slice the $u$ array and calculate each operation with one command:

这是预期的结果，执行时间几乎是瞬时的。如果我们以数组操作的方式执行同样的操作，那么与其单独计算 $u^n_i-u^n_{i-1}$ 5次，不如将 $u$ 数组切片，用一条命令计算每个操作。

In [3]:
u[1:] - u[0:-1]

array([1, 1, 1, 1, 1])

> What this command says is subtract the 0th, 1st, 2nd, 3rd, 4th and 5th elements of $u$ from the 1st, 2nd, 3rd, 4th, 5th and 6th elements of $u$.

这个命令的意思是用$u$的第0、1、2、3、4、5个元素减去$u$的第1、2、3、4、5、6个元素。

### Speed Increases / 速度提升

> For a 6 element array, the benefits of array operations are pretty slim. There will be no appreciable difference in execution time because there are so few operations taking place. But if we revisit 2D linear convection, we can see some substantial speed increases.

对于一个6个元素的数组，数组操作的好处是非常小的。在执行时间上不会有明显的差别，因为发生的操作太少了。但是如果我们重新审视二维线性对流，我们可以看到一些实质性的速度提升。

In [4]:
nx = 81
ny = 81
nt = 100
c = 1
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = .2
dt = sigma * dx

x = numpy.linspace(0,2,nx)
y = numpy.linspace(0,2,ny)

u = numpy.ones((ny,nx)) ## create a 1xn vector of 1's / 创建全为 1 的 1xn 向量
un = numpy.ones((ny,nx))

## Assign initial conditions / 分配初始条件

u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2

> With our initial conditions all set up, let's first try running our original nested loop code, making use of the iPython "magic" function `%%timeit`, which will help us evaluate the performance of our code.

设置好初始条件后，让我们首先尝试运行原始的嵌套循环代码，利用 iPython “magic” 函数 `%%timeit`，这将有助于我们评估代码的性能。

> **Note:** The `%%timeit` magic function will run the code several times and then give an average execution time as a result. If you have any figures being plotted within a cell where you run `%%timeit`, it will plot those figures repeatedly which can be a bit messy.

**注意：** `%%timeit` 函数将多次运行代码，然后给出平均执行时间作为结果。如果您在运行 `%%timeit` 的单元格中绘制了任何图形，它将重复绘制这些图形，这可能会有点混乱。

> The execution times below will vary from machine to machine. Don't expect your times to match these times, but you should expect to see the same general trend in decreasing execution time as we switch to array operations.

下面的执行时间因计算机而异。不要指望您的时间与这些时间相匹配，但当我们切换到数组操作时，您应该期望看到减少执行时间的相同总体趋势。

In [5]:
%%timeit
u = numpy.ones((ny,nx))
u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2

for n in range (nt + 1): ## loop across number of time steps / 跨多个时间步长循环
    un = u.copy()
    row, col = u.shape
    for j in range (1, row):
        for i in range (1, col):
            u[j,i] = (un[j,i] - (c * dt / dx * (un[j,i] - un[j,i-1])) - (c * dt / dy * (un[j,i] - un[j-1,i])))
            u[0,:] = 1
            u[-1,:] = 1
            u[:,0] = 1
            u[:,-1] = 1


996 ms ± 12.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


> With the "raw" Python code above, the mean execution time achieved was 0.975 seconds (on a Macbook Air with M1 chip). Keep in mind that with these three nested loops, that the statements inside the **j** loop are being evaluated more than 650,000 times. Let's compare that with the performance of the same code implemented with array operations:

使用上面的“原始”Python代码，实现的平均执行时间为0.975秒（在带有M1芯片的Macbook Air上）。请记住，使用这三个嵌套循环时，**j** 循环中的语句将被计算超过 650，000 次。让我们将其与使用数组操作实现的相同代码的性能进行比较：

In [6]:
%%timeit
u = numpy.ones((ny,nx))
u[int(.5 / dy):int(1 / dy + 1),int(.5 / dx):int(1 / dx + 1)] = 2

for n in range (nt + 1): ## loop across number of time steps / 跨多个时间步长循环
    un = u.copy()
    u[1:,1:] = (un[1:,1:] - (c * dt / dx * (un[1:,1:] - un[1:,0:-1])) - (c * dt / dy * (un[1:,1:] - un[0:-1,1:])))
    u[0,:] = 1
    u[-1,:] = 1
    u[:,0] = 1
    u[:,-1] = 1

2.76 ms ± 37.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


> As you can see, the speed increase is substantial. The same calculation goes from 0.975 seconds to 2.71 milliseconds. 1 second isn't a huge amount of time to wait, but these speed gains will increase exponentially with the size and complexity of the problem being evaluated.

如您所见，速度提升是显着的。相同的计算从 0.975 秒到 2.71 毫秒。1 秒并不是很长的等待时间，但随着所评估问题的规模和复杂性，这些速度提升将呈指数级增长。